In [1]:
import os
import sys
import time
import math
import serial
import threading
import numpy as np
from numpy import fft
from numpy import array
from sklearn import svm
from pickle import BINSTRING
from sklearn import grid_search
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib
#Object dectection Yolo CNN
import cv2
sys.path.insert(0, '/home/rahulsingh/Desktop/earble/yolo')
from darkflow.net.build import TFNet

##########################################################################################################################
#Step by step debugging in iPython: https://stackoverflow.com/questions/16867347/step-by-step-debugging-with-ipython
#Multi-threading in python: https://stackoverflow.com/questions/31768865/how-to-use-threading-to-get-user-input-realtime-while-main-still-running-in-pyth
##########################################################################################################################

options = {"model": "cfg/yolo.cfg", "load": "yolo/bin/yolo.weights", "threshold": 0.2}
tfnet = TFNet(options)


/home/rahulsingh/anaconda3/envs/pyproj3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/rahulsingh/anaconda3/envs/pyproj3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading yolo/bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.1176450252532959s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  

In [2]:
def get_frequencies(data, timestamps):
    num_seconds = float(timestamps[-2] - timestamps[0])
    samples_per_second = len(data) / num_seconds
    num_samples = len(data)
    oscilations_per_sample = [float(oscilations) / num_samples for oscilations in range(0, num_samples)]
    return [ops * samples_per_second for ops in oscilations_per_sample]

def get_buckets(data, first, last, num_buckets, hertz_cutoff=float(5)):
    # Transform all of the original data to be a single component along the first principal component
    pca = PCA(n_components=1, copy=True, whiten=True)
    transformed_dataset = PCA.fit_transform(pca, data)
    #print(pca.explained_variance_ratio_)
    slice=transformed_dataset[first:last]

    transformed = fft.fft(slice)
    absolute = [abs(complex) for complex in transformed]

    frequencies = get_frequencies(data, (data.T)[0])

    buckets = [0 for i in range(num_buckets)]
    width = hertz_cutoff / num_buckets
    sum_of_buckets = 0.0000001
    for i in range(1, len(absolute)):
        index = int(frequencies[i] / width)
        if index >= num_buckets:
            break;
        buckets[index] += absolute[i]
        sum_of_buckets += absolute[i]

    if 0: #arguments['--normalize']:
        buckets = map(lambda x: x/sum_of_buckets, buckets)

    return buckets

def get_samples(data, target):
    result = []
    intent = []
    segmentsize=30
    stride=5            # Reduce this to very little to get very large trainingsets
    noOfBuckets=40
    clas = np.array(target)
    for  start in range(0, len(data) - segmentsize, stride):
        if ((sum(clas[start: start + stride])<stride) & (sum(clas[start: start + stride])>0)): #((clas[start: start + stride]==).all()):
            continue
        if start + segmentsize <= len(data):
            segments_buckets = get_buckets(data, start, start + segmentsize, noOfBuckets)
            #print(segments_buckets)
            intent.append(clas[start])
            result.append(segments_buckets)
    return result, intent

def loadData(filename):
    return np.loadtxt(filename, delimiter=",")

def getPCASample(data, target):
    result = []
    intent = []
    segmentsize=30
    stride=2            # Reduce this to very little to get very large trainingsets
    noOfBuckets=40
    clas = np.array(target)
    pca = PCA(n_components=1, copy=True, whiten=True)
    
    #Pre-processing the data
    arr = np.array(data).T
    new_arr = arr[1:13]
    new_arr[0]  = (new_arr[0]*2)/ 32768.0
    new_arr[1]  = (new_arr[1]*2)/ 32768.0
    new_arr[2]  = (new_arr[2]*2)/ 32768.0
    new_arr[3]  = (new_arr[3]*250)/ 32768.0
    new_arr[4]  = (new_arr[4]*250)/ 32768.0
    new_arr[5]  = (new_arr[5]*250)/ 32768.0
    new_arr[6]  = (new_arr[6]*2)/ 32768.0
    new_arr[7]  = (new_arr[7]*2)/ 32768.0
    new_arr[8]  = (new_arr[8]*2)/ 32768.0
    new_arr[9]  = (new_arr[9]*250)/ 32768.0
    new_arr[10] = (new_arr[10]*250)/ 32768.0
    new_arr[11] = (new_arr[11]*250)/ 32768.0
    
    new_arr = new_arr.T
    
    transformed_dataset = PCA.fit_transform(pca, new_arr)
    #transformed_dataset = PCA.fit_transform(pca, data)
    
    for  start in range(0, len(data) - segmentsize, stride):
        if ((sum(clas[start: start + stride])<stride) & (sum(clas[start: start + stride])>0)): #((clas[start: start + stride]==).all()):
            continue
        if start + segmentsize <= len(data):
            intent.append(clas[start])
            result.append(np.reshape(transformed_dataset[start:start+segmentsize], (30,)))
    return result, intent

In [ ]:
#Create a list/dictionary of known recognizable objects using yolo
#Make a seperate counter for all of them
#And threshold values for each
#Convert counter into probability score

#Open serial port for reading data from Intel Curie IMU
ser0 = serial.Serial('/dev/ttyACM0', 115200) # Establish the connection on a specific port 0
ser1 = serial.Serial('/dev/ttyACM1', 115200) # Establish the connection on a specific port 1
ser0.flush()
ser1.flush()

#If want to train from imu data saved in file
load = True
if load:
    clf = joblib.load('1s_6sps.pkl')
    print(clf)
else:
    data, target = get_samples(np.array(imu), intent0)
    svr = svm.SVC()
    exponential_range = [pow(10, i) for i in range(-4, 1)]
    parameters = {'kernel':['linear', 'rbf'], 'C':exponential_range, 'gamma':exponential_range}
    clf = grid_search.GridSearchCV(svr, parameters, n_jobs=8, verbose=True)
    clf.fit(data, target)
    joblib.dump(clf, '1s_6sps.pkl')  #'../models/1s_6sps.pkl')
    print(clf)

camera = cv2.VideoCapture(1)
result = list()
counter= 0
flag   = 0
threshold  = 10
intent_thres_imu = 10
imu_data   = []
pred_count = 0
try:
    while True:
        ret, image = camera.read()
        cv2.imshow('frame', image)
        if(cv2.waitKey(1) & 0xFF == ord('q')):
            break
        out = (tfnet.return_predict(image))
        #increment/decrement the counter ==> probably in logrithmic way
        #then move to classifier SVM algorithm using IMU data
        result.append(out)
        for obj in out:
            if((obj["label"]=="bottle")==True):            #Check location of bottle as well to see if the bottle at particular location is detected
                flag = 1
                
        if(flag):
            counter = counter + 1
            flag = 0
            print("bottle counter: ", counter)
        else:
            if(counter>0):
                counter = counter - 1
                
        if(counter > threshold):
            print("Intention Probably is to pick up the bottle")
            #Do something with counter
            while True: #as of now continous loop but should be more adaptive and intuitive
                data_count = 0
                ser0.flush()
                ser1.flush()
                while(data_count<30):       #read data from imu
                    data0 = ser0.readline().rstrip()
                    data0 = data0.split()
                    data1 = ser1.readline().rstrip()
                    data1 = data1.split()
                    if (len(data0) == 6 & len(data1) == 6):
                        imu_data.append([(int(data0[0])*2.0)/ 32768.0, (int(data0[1])*2.0)/ 32768.0, (int(data0[2])*2.0)/ 32768.0, (int(data0[3])*250.0)/32768.0, (int(data0[4])*250.0)/32768.0, (int(data0[5])*250.0)/32768.0])
                        #imu_data.append([float(time.time()),  int(data0[0]), int(data0[1]), int(data0[2]), int(data0[3]), int(data0[4]), int(data0[5]), int(data1[0]), int(data1[1]), int(data1[2]), int(data1[3]), int(data1[4]), int(data1[5])])
                        data_count = data_count + 1

                pca = PCA(n_components=1, copy=True, whiten=True)
                transformed_dataset = PCA.fit_transform(pca, np.array(imu_data[-30:]))
                predicted = clf.predict(transformed_dataset.T)
                
                #bucket    = get_buckets(np.array(imu_data[-30:]), 1, 30, 40)   #send the last 30 elements data to get_samples function
                #predicted = clf.predict(np.reshape(bucket, (1, 40)))           #reshaping and sending for classification
                if(predicted == 1):
                    print("IMU intent Prediction::: Positive Count", pred_count)
                    pred_count = pred_count + 1
                    if(pred_count>intent_thres_imu):
                        print("Intended Action is XYZ")
                else:
                    print("IMU intent Prediction::: Negative Count", pred_count)
                    pred_count = pred_count - 1
        
except KeyboardInterrupt:
    camera.release()
    cv2.destroyAllWindows()
    ser0.close()
    ser1.close()
    pass
    #del(camera)

In [ ]:
inp = 0
#Using thread to store the user input for task classification
def background():
    global inp
    while True:
        inp = int(input())
        
#Now threading1 runs regardless of user input
threading1 = threading.Thread(target=background)
threading1.daemon = True
threading1.start()

ser0 = serial.Serial('/dev/ttyACM0', 115200) # Establish the connection on a specific port
ser1 = serial.Serial('/dev/ttyACM1', 115200) # Establish the connection on a specific port
counter0 = 0 
counter1 = 0 
fhandle  = open("imu.txt", 'ab')
fhandle0 = open("imu0.txt", 'ab')
fhandle1 = open("imu1.txt", 'ab')
ihandle0 = open("intent0.txt", 'ab')
ihandle1 = open("intent1.txt", 'ab')
imu0    = []  #contains tstamps with imu0 data
imu1    = []  #contains tstamps with imu1 data
imu     = []

intent0 = []
intent1 = []
ser0.flush()
ser1.flush()
start0 = time.time()
start1 = time.time()
t0     = time.time()
t1     = time.time()

#FFT might have the User Activity information as well i.e. running, walking, sitting etc. which can also be used for counterchecks for intent classification
#Or the above info can be captured as well in the SVM classifier
while True:
    try:
        counter0 +=1
        counter1 +=1
        #ser.write(str(chr(counter))) # Convert the decimal number to ASCII then send it to the Arduino
        data0 = ser0.readline().rstrip()
        data0 = data0.split()
        data1 = ser1.readline().rstrip()
        data1 = data1.split()
        if ((len(data0) == 6) & (len(data1) == 6) & (inp<2)):
            imu0.append([float(time.time()), int(data0[0]), int(data0[1]), int(data0[2]), int(data0[3]), int(data0[4]), int(data0[5])])
            imu1.append([float(time.time()), int(data1[0]), int(data1[1]), int(data1[2]), int(data1[3]), int(data1[4]), int(data1[5])])
            imu.append([float(time.time()),  (int(data0[0])*2.0)/ 32768.0, (int(data0[1])*2.0)/ 32768.0, (int(data0[2])*2.0)/ 32768.0, (int(data0[3])*250.0)/32768.0, (int(data0[4])*250.0)/32768.0, (int(data0[5])*250.0)/32768.0, (int(data1[0])*2.0)/ 32768.0, (int(data1[1])*2.0)/ 32768.0, (int(data1[2])*2.0)/ 32768.0, (int(data1[3])*250.0)/32768.0, (int(data1[4])*250.0)/32768.0, (int(data1[5])*250.0)/32768.0])
            intent0.append(inp)
            intent1.append(inp)
        if counter0 == 255:
            print("frequency:", 255/(time.time()-t0))
            t0 = time.time()
            counter0 = 0
        if counter1 == 255:
            print("frequency:", 255/(time.time()-t1))
            t1 = time.time()
            counter1 = 0
            
    except KeyboardInterrupt:
        end=time.time()
        d_time0=end-start0
        d_time1=end-start1
        np.savetxt(fhandle0, np.array(imu0),delimiter=",")
        fhandle0.close()
        np.savetxt(fhandle1, np.array(imu1),delimiter=",")
        fhandle1.close()
        np.savetxt(ihandle0, np.array(intent0),delimiter=",")
        ihandle0.close()
        np.savetxt(ihandle1, np.array(intent1),delimiter=",")
        ihandle1.close()
        np.savetxt(fhandle, np.array(imu),delimiter=",")
        fhandle.close()
        print("Avg Frequency is :: ", len(imu0)/d_time0)
        print("Avg Frequency is :: ", len(imu1)/d_time1)
        ser0.close()
        ser1.close()
        pass
        #exit(1)

In [22]:
###Naive Bayesian Approach
#Choose distributions that best characterize your data and prediction problem

from sklearn import datasets
iris = datasets.load_iris()
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
gnb = GaussianNB()

imuu = loadData("imu.txt")
inte= loadData("intent0.txt")
y_pred = gnb.fit(imuu, inte).predict(imuu)
print("Number of mislabeled points out of a total %d points : %d" % (imuu.shape[0],(inte != y_pred).sum()))
#Number of mislabeled points out of a total 150 points : 6


Number of mislabeled points out of a total 14160 points : 2005


In [18]:
y_pred.shape

(14160,)

In [23]:
#ser0 = serial.Serial('/dev/ttyACM0', 115200) # Establish the connection on a specific port 0
#ser1 = serial.Serial('/dev/ttyACM1', 115200) # Establish the connection on a specific port 1
#ser0.flush()
#ser1.flush()

from sklearn.model_selection import train_test_split

LOAD = False
SHOW_CONFUSION_MATRIX = True

intent_thres_imu = 10
imu_data   = []
pred_count = 0

imuu = loadData("imu.txt")
inte= loadData("intent0.txt")
if LOAD:
    clf = joblib.load('1s_6sps.pkl')
    print(clf)
else:
    #data, target = get_samples(np.array(imu), intent0)
    #data, target = getPCASample(imuu, inte)
    data  = imuu
    target= inte
    svr   = svm.SVC()
    exponential_range = [pow(10, i) for i in range(-4, 1)]
    parameters = {'kernel':['linear', 'rbf'], 'C':exponential_range, 'gamma':exponential_range}
    clf = grid_search.GridSearchCV(svr, parameters, n_jobs=8, verbose=True)
    
    X_train, X_test, Y_train, Y_test = train_test_split(data, target, test_size=0.35, random_state=0)
    clf.fit(X_train, Y_train)
    joblib.dump(clf, '1s_6sps.pkl')  #'../models/1s_6sps.pkl')
    print(clf)

if SHOW_CONFUSION_MATRIX:
    print("Confusion Matrix:")
    Y_predicted = clf.predict(X_test)
    print(confusion_matrix(Y_test, Y_predicted))
    
    print("\nBest estimator parameters: ")
    print(clf.best_estimator_)
    
    #Calculates the score of the best estimator found.
    score = clf.score(X_test, Y_test)
    
    print("\nSCORE: {score}\n".format(score = score))
    
    print("Saving the model...",)
    
    #Saves the model to the "model.pkl" file
    joblib.dump(clf, 'model.pkl') 
    #Saves the classes to the "classes.pkl" file
    #joblib.dump(classes, 'classes.pkl') 
    
    print("DONE")
    
"""
print("Intention Probably is to pick up the bottle")
data_count = 0
while(True):
    try:
        while(data_count<30):       #read data from imu
            data0 = ser0.readline().rstrip()
            data0 = data0.split()
            data1 = ser1.readline().rstrip()
            data1 = data1.split()
            if (len(data0) == 6 & len(data1) == 6):
                imu_data.append([float(time.time()),  int(data0[0]), int(data0[1]), int(data0[2]), int(data0[3]), int(data0[4]), int(data0[5]), int(data1[0]), int(data1[1]), int(data1[2]), int(data1[3]), int(data1[4]), int(data1[5])])
                data_count = data_count + 1
                
        data_count = 0
        bucket    = get_buckets(np.array(imu_data[-30:]), 1, 30, 40)   #send the last 30 elements data to get_samples function
        predicted = clf.predict(np.reshape(bucket, (1, 40)))     #reshaping and sending for classification
        print(bucket)
        if(predicted == 1):
            pred_count = pred_count + 1
            print("IMU intent Prediction Count", pred_count)
            if(pred_count>intent_thres_imu):
                print("Intended Action is XYZ")
            else:
                print("IMU intent Prediction Count", pred_count)
                pred_count = pred_count - 1
    except KeyboardInterrupt:
        ser0.flush()
        ser0.close()
        ser1.flush()
        ser1.close()
        pass
        #keep a counter for it as well
        #wait for threshold to predict the intection
"""

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  1.1min
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/rahulsingh/anaconda3/envs/pyproj3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-34906b94ac34>", line 31, in <module>
    clf.fit(X_train, Y_train)
  File "/home/rahulsingh/anaconda3/envs/pyproj3/lib/python3.6/site-packages/sklearn/grid_search.py", line 838, in fit
    return self._fit(X, y, ParameterGrid(self.param_grid))
  File "/home/rahulsingh/anaconda3/envs/pyproj3/lib/python3.6/site-packages/sklearn/grid_search.py", line 574, in _fit
    for parameters in parameter_iterable
  File "/home/rahulsingh/anaconda3/envs/pyproj3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 789, in __call__
    self.retrieve()
  File "/home/rahulsingh/anaconda3/envs/pyproj3/lib/python3.6/site-packages/sklearn/externals/joblib/parallel.py", line 699, in retrieve
    self._output.extend(job.get(timeout=self.timeout)

KeyboardInterrupt: 

In [ ]:
tmp = np.array(imuu)
a,b = getPCASample(imuu, inte)

In [26]:
#pca = PCA(n_components=1, copy=True, whiten=True)
#transformed_dataset = PCA.fit_transform(pca, imuu[0:30])
#predicted = clf.predict(transformed_dataset.T)
import numpy as np
from sklearn.naive_bayes import GaussianNB

imuu = loadData("imu.txt")
inte= loadData("intent0.txt")

clf = GaussianNB()
clf.fit(imuu, inte)
#GaussianNB(priors=None, var_smoothing=1e-09)
#print(clf.predict([[-0.8, -1]]))

#clf_pf = GaussianNB()
#clf_pf.partial_fit(X, Y, np.unique(Y))
#GaussianNB(priors=None, var_smoothing=1e-09)
#print(clf_pf.predict([[-0.8, -1]]))

predicted = clf.predict(imuu[900:930])
print(predicted)
#print(transformed_dataset)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
